## Camera calibration

Pinhole and thin-lens cameras accept OpenCV-like intrinsics matrix in setup/update functions. This notebook illustrates how intrinsic parameters should be provided and how you can recover them with OpenCV camera calibration.

In [1]:
import numpy as np
import cv2 as cv
from plotoptix import TkOptiX

Setup the raytracer:

In [2]:
rt = TkOptiX(start_now=False)

rt.set_param(min_accumulation_step=4,
             max_accumulation_frames=512)

Add chess board pattern used for calibration.

In [3]:
rx = 10
ry = 10
n = 11

x = np.linspace(0, rx, n)
y = np.linspace(0, ry, n)
X, Y = np.meshgrid(x, y)

# positions of cubes
xyz = np.stack((X.flatten(), Y.flatten(), np.zeros(n**2))).T - np.array([0, 0, 0.15])

widx = [i for i in range(0,xyz.shape[0]) if i % 2 == 0]
bidx = [i for i in range(0,xyz.shape[0]) if i % 2 == 1]

xp = np.linspace(1, rx, n-1)
yp = np.linspace(1, ry, n-1)
Xp, Yp = np.meshgrid(xp, yp)
xyzp = np.stack((Xp.flatten(), Yp.flatten(), np.zeros((n-1)**2))).T.astype(np.float32)
#rt.set_data("points", xyzp, r=0.1, c=0.9) # points to confirm true positions of objpoints

rt.set_data("wcubes", xyz[widx,:], u=[0.998, 0, 0], v=[0, 0.998, 0], w=[0, 0, 0.15], c=0.93, geom="Parallelepipeds")
rt.set_data("bcubes", xyz[bidx,:], u=[0.998, 0, 0], v=[0, 0.998, 0], w=[0, 0, 0.15], c=0.15, geom="Parallelepipeds")
rt.set_data("base", [-0.5, -0.5, -0.3], u=[12, 0, 0], v=[0, 12, 0], w=[0, 0, 0.2], c=0.9, geom="Parallelepipeds")

rt.set_data("plane", [-20, -20, -1], u=[50, 0, 0], v=[0, 50, 0], c=0.9, geom="Parallelograms") # a wall behind cubes

rt.setup_area_light("light1", center=[15, 4, 15], target=[5, 4, 0], u=7, v=7, color=[8.5, 8, 7.5])

rt.set_ambient([0.1, 0.2, 0.4])
rt.set_background(0)

Camera setup. Absolute scale in [mm] is used, though it requires to specify sensor height (Y dimension).

In [4]:
sensor_height = 24 # [mm]
fx = 17 # [mm]
fy = 17 # [mm]
cx = -3  # [mm]
cy = -1  # [mm]

# OpenCV-like camera intrinsic matrix
cam_mat = np.array([[fx, 0, cx], [0, fy, cy], [0, 0, 1]], dtype=np.float32)

eye = [7, 7, 10]
tgt = [6, 6, 0]
up  = [0, -1, 0]

rt.setup_camera(
    "cam1", cam_type="ThinLens",
    eye=eye, target=tgt, up=up,
    camera_matrix=cam_mat,
    sensor_height=sensor_height,
    glock=True
)

Start the ray tracing:

In [5]:
rt.start()

**Collect calibration images.**

Calibration with OpenCV is performed for a fixed image size, this it is set below to 1300x950 to avoid changes caused by the GUI.

Change the anlgle of view and/or camera target and wait until callback notifies image was captured. Collect a few images. These will be used to reconstruct camera parameters.

In [6]:
width = 1500
height = 950

rt.set_rt_size((width, height))

imgpoints = []
objpoints = []

def image_ready(rt: TkOptiX) -> None:
    gray = cv.cvtColor(rt._img_rgba, cv.COLOR_BGR2GRAY)
    retval, corners = cv.findChessboardCorners(gray, (n-1, n-1))
    if retval:
        corners2 = cv.cornerSubPix(
            gray, corners, (5,5), (-1,-1),
            (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 10, 0.001)
        )
        imgpoints.append(corners2)
        objpoints.append(100*xyzp.astype(np.float32))

        print(len(imgpoints), "images captured")
    else:
        print("skip image")
    
rt.set_accum_done_cb(image_ready)

1 images captured
2 images captured
3 images captured
4 images captured
5 images captured
6 images captured
7 images captured


Run camera calibration. OpenCV returns calues in [pixels] so they need to appropriate scaling to get values back in [mm].

In [7]:
img_size = rt.get_size()
sensor_width = sensor_height * width / height
pixel_pitch = sensor_height / img_size[1]

ret, mtx, dist, rvecs, tvecs = cv.calibrateCamera(objpoints, imgpoints, img_size, None, None)

print("fx:", mtx[0,0])
print("fy:", mtx[1,1])
print("fx:", pixel_pitch * mtx[0,0], "[mm]")
print("fy:", pixel_pitch * mtx[1,1], "[mm]")
print("cx:", mtx[0,2] / img_size[0] - 0.5)
print("cy:", mtx[1,2] / img_size[1] - 0.5)
print("cx:", sensor_height * (mtx[0,2] / img_size[0] - 0.5), "[mm]")
print("cy:", sensor_height * (mtx[1,2] / img_size[1] - 0.5), "[mm]")
print("dist:", dist)

fx: 672.8845337359755
fy: 672.8827714273825
fx: 16.99918822069833 [mm]
fy: 16.999143699218084 [mm]
cx: -0.12541289116206544
cy: -0.0422587992651956
cx: -3.0099093878895706 [mm]
cy: -1.0142111823646944 [mm]
dist: [[-4.85686172e-05  1.83963466e-04 -4.10056540e-06 -2.78562411e-05
  -7.12502877e-05]]


Try another set of parameters and re-run image capturing and calibration.

In [8]:
sensor_height = 24 # [mm]
fx = 21 # [mm]
fy = 18 # [mm]
cx = 0  # [mm]
cy = -2  # [mm]

# OpenCV-like camera intrinsic matrix
cam_mat = np.array([[fx, 0, cx], [0, fy, cy], [0, 0, 1]], dtype=np.float32)

rt.update_camera("cam1",
    camera_matrix=cam_mat,
    sensor_height=sensor_height,
)

8 images captured


Close the ray-tracer.

In [9]:
rt.close()